# 載入函式

In [ ]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import tensorflow as tf
import os # 因為要建立資料夾

In [ ]:
from tensorflow.keras.models import Sequential # CNN 建立空模函式
from tensorflow.keras.layers import Dense, Dropout, Flatten, Activation, Conv2D, MaxPooling2D # CNN 函式
from tensorflow.python.keras.utils import np_utils # 層

# 整理圖片

In [ ]:
melon_dir = '/content/drive/MyDrive/2023聯成電腦/Python IV 視覺辨識與機器學習/photos/水果30分類/train/哈密瓜'
kiwi_dir = '/content/drive/MyDrive/2023聯成電腦/Python IV 視覺辨識與機器學習/photos/水果30分類/train/奇異果'
# 改變圖片大小
re_melon = '/content/drive/MyDrive/2023聯成電腦/Python IV 視覺辨識與機器學習/photos/水果30分類/train/哈密瓜/resize'
re_kiwi = '/content/drive/MyDrive/2023聯成電腦/Python IV 視覺辨識與機器學習/photos/水果30分類/train/奇異果/resize'

In [ ]:
# 改成相同的圖檔格式
melon_path = os.path.join(re_melon)
kiwi_path = os.path.join(re_kiwi)

In [ ]:
file_name = '/content/drive/MyDrive/2023聯成電腦/Python IV 視覺辨識與機器學習/photos/水果30分類/train/哈密瓜/101.jpeg'
name = file_name.split('/')[-1]
print(name)

101.jpeg


## 抓取檔案

In [ ]:
import glob
from PIL import Image

# 抓取目錄下的所有jpg檔案
imgs = glob.glob(melon_dir + '/*.jpg')
imgs

['/content/drive/MyDrive/2023聯成電腦/Python IV 視覺辨識與機器學習/photos/水果30分類/train/哈密瓜/2.jpg',
 '/content/drive/MyDrive/2023聯成電腦/Python IV 視覺辨識與機器學習/photos/水果30分類/train/哈密瓜/3.jpg',
 '/content/drive/MyDrive/2023聯成電腦/Python IV 視覺辨識與機器學習/photos/水果30分類/train/哈密瓜/5.jpg',
 '/content/drive/MyDrive/2023聯成電腦/Python IV 視覺辨識與機器學習/photos/水果30分類/train/哈密瓜/6.jpg',
 '/content/drive/MyDrive/2023聯成電腦/Python IV 視覺辨識與機器學習/photos/水果30分類/train/哈密瓜/10.jpg',
 '/content/drive/MyDrive/2023聯成電腦/Python IV 視覺辨識與機器學習/photos/水果30分類/train/哈密瓜/12.jpg',
 '/content/drive/MyDrive/2023聯成電腦/Python IV 視覺辨識與機器學習/photos/水果30分類/train/哈密瓜/11.jpg',
 '/content/drive/MyDrive/2023聯成電腦/Python IV 視覺辨識與機器學習/photos/水果30分類/train/哈密瓜/17.jpg',
 '/content/drive/MyDrive/2023聯成電腦/Python IV 視覺辨識與機器學習/photos/水果30分類/train/哈密瓜/16.jpg',
 '/content/drive/MyDrive/2023聯成電腦/Python IV 視覺辨識與機器學習/photos/水果30分類/train/哈密瓜/19.jpg',
 '/content/drive/MyDrive/2023聯成電腦/Python IV 視覺辨識與機器學習/photos/水果30分類/train/哈密瓜/18.jpg',
 '/content/drive/MyDrive/2023聯成電腦/Python IV 視覺辨

## 製成縮圖

不能太大，否則CPU會被拉太高，效能會變差

In [ ]:
size = (250, 250)
for item in imgs:
  images = cv2.imread(item)
  # :2 兩通道
  h, w = images.shape[:2]
  # 找最大邊
  if h > size[0] or w > size[1]:
    # 找出最大比例 => 等比例縮小
    ratio = max(h/size[0], w/size[1])
    images = cv2.resize(images, (int(w/ratio), int(h/ratio)))
    # 重新抓取新的寬、高
  h, w = images.shape[:2]
  # 高度的黑邊
  pad_h = size[0]-h
  # 寬度的黑邊
  pad_w = size[1]-w
  # 上、下補黑邊的高度
  top, bottom = pad_h//2, pad_h-(pad_h//2)
  # 左、右補黑邊的寬度
  left, right = pad_w//2, pad_w-(pad_w//2)
  # 補黑邊(填充：適用於圖片縮小時)
  images = cv2.copyMakeBorder(images, top, bottom, left, right, cv2.BORDER_CONSTANT, value=[0,0,0])
  # 抓檔名
  name = item.split('/')[-1]
  # 用程式判斷是否有所需的資料夾
  if not os.path.exists(melon_path):
    os.makedirs(melon_path)
    # 寫檔
    # 方法一：自己建立一個resize資料夾，寫絕對路徑
    # Python 字串前面加u,r,b,f的含義  https://iter01.com/570250.html
    #cv2.imwrite(f"/content/drive/MyDrive/2023聯成電腦/Python IV 視覺辨識與機器學習/photos/水果30分類/train/哈密瓜/resize/{name}", images)
    # 方法二：用上面的參數代入
  cv2.imwrite(f"{re_melon}/{name}", images)

# 訓練

In [ ]:
x_train = list()
y_train = list()

def create_data(path, category):
  for p in os.listdir(path):
    # 轉灰階
    img_array = cv2.imread(os.path.join(path, p), 0)
    # 訓練題目
    x_train.append(img_array)
    # 訓練答案
    y_train.append(category)

In [ ]:
# 訓練哈密瓜
create_data(melon_path, 0)

In [ ]:
len(x_train)

132

In [ ]:
len(y_train)

132

## 重複的code(訓練不同項)

In [ ]:
import glob
from PIL import Image

# 抓取目錄下的所有jpg檔案
imgs = glob.glob(kiwi_dir + '/*.jpg')
imgs

['/content/drive/MyDrive/2023聯成電腦/Python IV 視覺辨識與機器學習/photos/水果30分類/train/奇異果/1.jpg',
 '/content/drive/MyDrive/2023聯成電腦/Python IV 視覺辨識與機器學習/photos/水果30分類/train/奇異果/3.jpg',
 '/content/drive/MyDrive/2023聯成電腦/Python IV 視覺辨識與機器學習/photos/水果30分類/train/奇異果/2.jpg',
 '/content/drive/MyDrive/2023聯成電腦/Python IV 視覺辨識與機器學習/photos/水果30分類/train/奇異果/5.jpg',
 '/content/drive/MyDrive/2023聯成電腦/Python IV 視覺辨識與機器學習/photos/水果30分類/train/奇異果/7.jpg',
 '/content/drive/MyDrive/2023聯成電腦/Python IV 視覺辨識與機器學習/photos/水果30分類/train/奇異果/8.jpg',
 '/content/drive/MyDrive/2023聯成電腦/Python IV 視覺辨識與機器學習/photos/水果30分類/train/奇異果/10.jpg',
 '/content/drive/MyDrive/2023聯成電腦/Python IV 視覺辨識與機器學習/photos/水果30分類/train/奇異果/9.jpg',
 '/content/drive/MyDrive/2023聯成電腦/Python IV 視覺辨識與機器學習/photos/水果30分類/train/奇異果/11.jpg',
 '/content/drive/MyDrive/2023聯成電腦/Python IV 視覺辨識與機器學習/photos/水果30分類/train/奇異果/12.jpg',
 '/content/drive/MyDrive/2023聯成電腦/Python IV 視覺辨識與機器學習/photos/水果30分類/train/奇異果/14.jpg',
 '/content/drive/MyDrive/2023聯成電腦/Python IV 視覺辨識與機

In [ ]:
size = (250, 250)
for item in imgs:
  images = cv2.imread(item)
  # :2 兩通道
  h, w = images.shape[:2]
  # 找最大邊
  if h > size[0] or w > size[1]:
    # 找出最大比例 => 等比例縮小
    ratio = max(h/size[0], w/size[1])
    images = cv2.resize(images, (int(w/ratio), int(h/ratio)))
    # 重新抓取新的寬、高
  h, w = images.shape[:2]
  # 高度的黑邊
  pad_h = size[0]-h
  # 寬度的黑邊
  pad_w = size[1]-w
  # 上、下補黑邊的高度
  top, bottom = pad_h//2, pad_h-(pad_h//2)
  # 左、右補黑邊的寬度
  left, right = pad_w//2, pad_w-(pad_w//2)
  # 補黑邊(填充：適用於圖片縮小時)
  images = cv2.copyMakeBorder(images, top, bottom, left, right, cv2.BORDER_CONSTANT, value=[0,0,0])
  # 抓檔名
  name = item.split('/')[-1]
  # 用程式判斷是否有所需的資料夾
  if not os.path.exists(kiwi_path):
    os.makedirs(kiwi_path)
    # 寫檔
    # 方法一：自己建立一個resize資料夾，寫絕對路徑
    # Python 字串前面加u,r,b,f的含義  https://iter01.com/570250.html
    #cv2.imwrite(f"/content/drive/MyDrive/2023聯成電腦/Python IV 視覺辨識與機器學習/photos/水果30分類/train/哈密瓜/resize/{name}", images)
    # 方法二：用上面的參數代入
  cv2.imwrite(f"{re_kiwi}/{name}", images)

In [ ]:
# 訓練奇異果
create_data(kiwi_path, 1)

In [ ]:
len(x_train)


263

## 打散成組的資料

In [ ]:
# reshape(-1,250,250,1) 一組資料一維改成一個資料一維
x_train = np.array(x_train).reshape(-1,250,250,1) # (張數, 長, 寬, 通道數)
y_train = np.array(y_train)

In [ ]:
x_train.shape

(263, 250, 250, 1)

In [ ]:
x_train[0]

array([[[0],
        [0],
        [0],
        ...,
        [0],
        [0],
        [0]],

       [[0],
        [0],
        [0],
        ...,
        [0],
        [0],
        [0]],

       [[0],
        [0],
        [0],
        ...,
        [0],
        [0],
        [0]],

       ...,

       [[0],
        [0],
        [0],
        ...,
        [0],
        [0],
        [0]],

       [[0],
        [0],
        [0],
        ...,
        [0],
        [0],
        [0]],

       [[0],
        [0],
        [0],
        ...,
        [0],
        [0],
        [0]]], dtype=uint8)

In [ ]:
y_train

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [ ]:
x_train = x_train / 255 # 色階 => 收斂
# 將答案分類 OneHot
y_train = np_utils.to_categorical(y_train)

In [ ]:
y_train[0]

array([1., 0.], dtype=float32)

## reshape 用法

In [ ]:
n = np.arange(6)
n

array([0, 1, 2, 3, 4, 5])

In [ ]:
n = n.reshape(3,2)
n

array([[0, 1],
       [2, 3],
       [4, 5]])

# 建模

In [ ]:
model = Sequential()

In [ ]:
# 卷積
# filters=32 特徵圖 => 濾波器數目，就是下圖的每一階段的輸出面數或是深度，輸出的圖稱為『特徵圖』(Feature Map)，通常是4的倍數。
# kernel_size=(3,3) 卷機核的大小
# padding='same' 超過邊界時一律補0
# input_shape=x_train.shape[1:] 從索引值1開始抓到最後(張數不要)
# activation='relu' 修正線性單元激勵函式 (把負數變為0)
model.add(Conv2D(filters=32, kernel_size=(3,3), padding='same', input_shape=x_train.shape[1:], activation='relu'))

In [ ]:
# 池化
model.add(MaxPooling2D(pool_size=(2,2))) # pool_size=(2,2) 為預設，不寫也可以
# 卷積
model.add(Conv2D(filters=64, kernel_size=(3,3), padding='same', activation='relu'))
# 池化
model.add(MaxPooling2D(pool_size=(2,2)))

In [ ]:
# 放棄的比率
model.add(Dropout(0.2))
# 扁平化
model.add(Flatten())
# 放棄的比率
model.add(Dropout(0.2))
# 連接層：通常用1024(三通道 32*32*32)
model.add(Dense(1024, activation='relu'))
# 輸出層：2種分類
model.add(Dense(2, activation='softmax'))

# 編譯(程式)

In [ ]:
# optimizer='adam' 優化器
# loss 損失函式 ='mse' 平均方差
# metrics='accurancy' 只顯示正確度
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# 訓練

In [ ]:
#train_history = model.fit(x_train, y_train, batch_size=128, epochs=30)

# 作業

請用原本的x_train 和 y_train 來fit，看準確率共多少(幾成) y_test, x_test 去測

請打散 x_train, y_train 再做測試

請比較使用彩色與灰階的準確率

## 打亂資料群

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
# test_size=1 測試1筆
rX_train, rX_test, ry_train, ry_test = train_test_split(x_train, y_train, test_size=1, random_state=1)

In [ ]:
model.fit(rX_train, ry_train, epochs=10, batch_size=50, validation_split=0.25)

Epoch 1/10
4/4 [==============================] - 1s 239ms/step - loss: 0.4498 - accuracy: 0.8827 - val_loss: 0.8383 - val_accuracy: 0.4394
Epoch 2/10
4/4 [==============================] - 1s 207ms/step - loss: 0.3598 - accuracy: 0.8418 - val_loss: 0.8723 - val_accuracy: 0.5909
Epoch 3/10
4/4 [==============================] - 1s 208ms/step - loss: 0.2644 - accuracy: 0.9286 - val_loss: 1.4443 - val_accuracy: 0.4848
Epoch 4/10
4/4 [==============================] - 1s 217ms/step - loss: 0.4277 - accuracy: 0.7704 - val_loss: 0.9460 - val_accuracy: 0.5152
Epoch 5/10
4/4 [==============================] - 1s 214ms/step - loss: 0.2355 - accuracy: 0.9286 - val_loss: 1.0727 - val_accuracy: 0.5455
Epoch 6/10
4/4 [==============================] - 1s 219ms/step - loss: 0.2057 - accuracy: 0.9541 - val_loss: 1.1198 - val_accuracy: 0.5152
Epoch 7/10
4/4 [==============================] - 1s 215ms/step - loss: 0.1487 - accuracy: 0.9796 - val_loss: 1.3881 - val_accuracy: 0.4848
Epoch 8/10
4/4 [====

## 製作測試的資料

In [ ]:
melon_dir = '/content/drive/MyDrive/2023聯成電腦/Python IV 視覺辨識與機器學習/photos/水果30分類/test/哈密瓜'
kiwi_dir = '/content/drive/MyDrive/2023聯成電腦/Python IV 視覺辨識與機器學習/photos/水果30分類/test/奇異果'
# 改變圖片大小
re_melon = '/content/drive/MyDrive/2023聯成電腦/Python IV 視覺辨識與機器學習/photos/水果30分類/test/哈密瓜/resize'
re_kiwi = '/content/drive/MyDrive/2023聯成電腦/Python IV 視覺辨識與機器學習/photos/水果30分類/test/奇異果/resize'

In [ ]:
# 改成相同的圖檔格式
melon_path = os.path.join(re_melon)
kiwi_path = os.path.join(re_kiwi)

In [ ]:
import glob
from PIL import Image

# 抓取目錄下的所有jpg檔案
imgs = glob.glob(kiwi_dir + '/*.jpg') # melon 改成 kiwi 再跑一次

# 製成縮圖
size = (250, 250)
for item in imgs:
  images = cv2.imread(item)
  # :2 兩通道
  h, w = images.shape[:2]
  # 找最大邊
  if h > size[0] or w > size[1]:
    # 找出最大比例 => 等比例縮小
    ratio = max(h/size[0], w/size[1])
    images = cv2.resize(images, (int(w/ratio), int(h/ratio)))
    # 重新抓取新的寬、高
  h, w = images.shape[:2]
  # 高度的黑邊
  pad_h = size[0]-h
  # 寬度的黑邊
  pad_w = size[1]-w
  # 上、下補黑邊的高度
  top, bottom = pad_h//2, pad_h-(pad_h//2)
  # 左、右補黑邊的寬度
  left, right = pad_w//2, pad_w-(pad_w//2)
  # 補黑邊(填充：適用於圖片縮小時)
  images = cv2.copyMakeBorder(images, top, bottom, left, right, cv2.BORDER_CONSTANT, value=[0,0,0])
  # 抓檔名
  name = item.split('/')[-1]
  # 用程式判斷是否有所需的資料夾
  if not os.path.exists(kiwi_path):
    os.makedirs(kiwi_path)
    # 寫檔
    # 方法一：自己建立一個resize資料夾，寫絕對路徑
    # Python 字串前面加u,r,b,f的含義  https://iter01.com/570250.html
    #cv2.imwrite(f"/content/drive/MyDrive/2023聯成電腦/Python IV 視覺辨識與機器學習/photos/水果30分類/train/哈密瓜/resize/{name}", images)
    # 方法二：用上面的參數代入
  cv2.imwrite(f"{re_kiwi}/{name}", images)


In [ ]:
x_test = list()
y_test = list()

def create_testdata(path, category):
  for p in os.listdir(path):
    # 轉灰階
    img_array = cv2.imread(os.path.join(path, p), 0)
    # 訓練題目
    x_test.append(img_array)
    # 訓練答案
    y_test.append(category)

In [ ]:
# 測試哈密瓜
create_testdata(melon_path, 0)

In [ ]:
# 測試哈密瓜
create_testdata(kiwi_path, 1)

In [ ]:
# reshape(-1,250,250,1) 一組資料一維改成一個資料一維
x_test = np.array(x_test).reshape(-1,250,250,1) # (張數, 長, 寬, 通道數)
y_test = np.array(y_test)

In [ ]:
x_test = x_test / 255 # 色階 => 收斂
# 將答案分類 OneHot
y_test = np_utils.to_categorical(y_test)

In [ ]:
# 打散測試資料
raX_train, raX_test, ray_train, _ = train_test_split(x_test, y_test, test_size=1, random_state=1)

## 測試

In [ ]:
predict1 = model.predict(x_test)
predict2 = model.predict(raX_train)

2/2 [==============================] - 0s 21ms/step


In [ ]:
right = 0
for i in range(len(predict1)):
  ans = np.argmax(predict1[i])
  pre = y_test[i][ans]
  if pre == 1:
    right += 1
right / len(predict1)

0.6190476190476191

In [ ]:
right = 0
for i in range(len(predict2)):
  ans = np.argmax(predict2[i])
  pre = ray_train[i][ans]
  if pre == 1:
    right += 1
right / len(predict2)

0.6129032258064516